In [3]:
import os
from dotenv import load_dotenv # requires python-dotenv
load_dotenv("./../config/credentials_my.env")

print(f'Project Connection String: <{os.environ["PROJECT_CONNECTION_STRING"][:30]}...>')

Project Connection String: <eastus.api.azureml.ms;eca2eddb...>


In [4]:
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential

project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(), conn_str=os.environ["PROJECT_CONNECTION_STRING"]
)

project_client.scope

{'subscription_id': 'eca2eddb-0f0c-4351-a634-52751499eeea',
 'resource_group_name': 'mmaiagent01rg',
 'project_name': 'aiagenthub01prj01-a6zz'}

In [5]:
from azure.ai.projects.models import CodeInterpreterTool

code_interpreter = CodeInterpreterTool()

print(f"Code interpreter definitions: {code_interpreter.definitions}")
print(f"Code interpreter resources: {code_interpreter.resources}")
print(f"Code interpreter file_ids: {code_interpreter.file_ids}")

Code interpreter definitions: [{'type': 'code_interpreter'}]
Code interpreter resources: {}
Code interpreter file_ids: set()


In [6]:
agent = project_client.agents.create_agent(
    model="gpt-4o-mini", # gpt-4o-mini or gpt-4o
    name="my-agent",
    instructions="You are a helpful agent",
    tools=code_interpreter.definitions,
    tool_resources=code_interpreter.resources
)

agent.items

<bound method _MyMutableMapping.items of {'id': 'asst_RFBLqCOc6t5wct92nS7bIqhK', 'object': 'assistant', 'created_at': 1735516750, 'name': 'my-agent', 'description': None, 'model': 'gpt-4o-mini', 'instructions': 'You are a helpful agent', 'tools': [{'type': 'code_interpreter'}], 'top_p': 1.0, 'temperature': 1.0, 'tool_resources': {'code_interpreter': {'file_ids': []}}, 'metadata': {}, 'response_format': 'auto'}>

# Create a second agent
agent_02 = project_client.agents.create_agent(
    model="gpt-4o", # gpt-4o-mini or gpt-4o
    name="my-agent",
    instructions="You are a helpful agent",
    tools=code_interpreter.definitions,
    tool_resources=code_interpreter.resources,
)

agent_02.items

In [7]:
# List all agents
print(project_client.agents.list_agents()['data'])

[{'id': 'asst_RFBLqCOc6t5wct92nS7bIqhK', 'object': 'assistant', 'created_at': 1735516750, 'name': 'my-agent', 'description': None, 'model': 'gpt-4o-mini', 'instructions': 'You are a helpful agent', 'tools': [{'type': 'code_interpreter'}], 'top_p': 1.0, 'temperature': 1.0, 'tool_resources': {'code_interpreter': {'file_ids': []}}, 'metadata': {}, 'response_format': 'auto'}]


project_client.agents.delete_agent(agent_02.id)

In [8]:
# Create a thread
thread = project_client.agents.create_thread()

print(f"Created thread: {thread}")

Created thread: {'id': 'thread_wOlIdhZDPEmOqMBIih8MnxJF', 'object': 'thread', 'created_at': 1735516762, 'metadata': {}, 'tool_resources': {}}


In [9]:
# Create a message
message = project_client.agents.create_message(
    thread_id=thread.id,
    role="user",
    content="""
    Could you please create a bar chart for the operating profit using the following data and provide the file to me? 
    Company A: $1.2 million, Company B: $2.5 million, Company C: $3.0 million, Company D: $1.8 million
    """,
)
print(f"Created message: {message}")

Created message: {'id': 'msg_FRag7j794rIbmaVI6WwRRHMO', 'object': 'thread.message', 'created_at': 1735516763, 'assistant_id': None, 'thread_id': 'thread_wOlIdhZDPEmOqMBIih8MnxJF', 'run_id': None, 'role': 'user', 'content': [{'type': 'text', 'text': {'value': '\n    Could you please create a bar chart for the operating profit using the following data and provide the file to me? \n    Company A: $1.2 million, Company B: $2.5 million, Company C: $3.0 million, Company D: $1.8 million\n    ', 'annotations': []}}], 'attachments': [], 'metadata': {}}


In [10]:
from azure.ai.projects.models import FilePurpose
#upload a file
file = project_client.agents.upload_file(file_path='./product_info_1.md', purpose="assistants")
print(f"Uploaded file, file ID: {file.id}")

Uploaded file, file ID: assistant-RN606WB5R2TuupaxQsAVXOLj


In [11]:
%%time
# import logging
# logging.basicConfig(level=logging.INFO) # Configure logging 

# Run the agent
run = project_client.agents.create_and_process_run(thread_id=thread.id, assistant_id=agent.id)

print(f"Run status: {run.status}.\nRun details: {run}")

Run status: RunStatus.COMPLETED.
Run details: {'id': 'run_xTDulmZGvvNgMlRyMzszYB2j', 'object': 'thread.run', 'created_at': 1735516772, 'assistant_id': 'asst_RFBLqCOc6t5wct92nS7bIqhK', 'thread_id': 'thread_wOlIdhZDPEmOqMBIih8MnxJF', 'status': 'completed', 'started_at': 1735516773, 'expires_at': None, 'cancelled_at': None, 'failed_at': None, 'completed_at': 1735516780, 'required_action': None, 'last_error': None, 'model': 'gpt-4o-mini', 'instructions': 'You are a helpful agent', 'tools': [{'type': 'code_interpreter'}], 'tool_resources': {}, 'metadata': {}, 'temperature': 1.0, 'top_p': 1.0, 'max_completion_tokens': None, 'max_prompt_tokens': None, 'truncation_strategy': {'type': 'auto', 'last_messages': None}, 'incomplete_details': None, 'usage': {'prompt_tokens': 408, 'completion_tokens': 222, 'total_tokens': 630}, 'response_format': 'auto', 'tool_choice': 'auto', 'parallel_tool_calls': True}
CPU times: total: 15.6 ms
Wall time: 7.84 s


In [12]:
run.status

<RunStatus.COMPLETED: 'completed'>

In [13]:
# Get messages from the thread
messages = project_client.agents.list_messages(thread_id=thread.id)
print(f"Messages: {messages}")

Messages: {'object': 'list', 'data': [{'id': 'msg_9LH2KWDwmWSw6mCcG3VuPQyI', 'object': 'thread.message', 'created_at': 1735516779, 'assistant_id': 'asst_RFBLqCOc6t5wct92nS7bIqhK', 'thread_id': 'thread_wOlIdhZDPEmOqMBIih8MnxJF', 'run_id': 'run_xTDulmZGvvNgMlRyMzszYB2j', 'role': 'assistant', 'content': [{'type': 'image_file', 'image_file': {'file_id': 'assistant-oyhGeEMWiaxncjaak161lDMQ'}}, {'type': 'text', 'text': {'value': 'I have created the bar chart for the operating profit of the companies. You can download it using the link below:\n\n[Download the bar chart](sandbox:/mnt/data/operating_profit_bar_chart.png)', 'annotations': [{'type': 'file_path', 'text': 'sandbox:/mnt/data/operating_profit_bar_chart.png', 'start_index': 140, 'end_index': 188, 'file_path': {'file_id': 'assistant-H1xD6bD84ahBz155QIPDZl5v'}}]}}], 'attachments': [{'file_id': 'assistant-H1xD6bD84ahBz155QIPDZl5v', 'tools': [{'type': 'code_interpreter'}]}], 'metadata': {}}, {'id': 'msg_FRag7j794rIbmaVI6WwRRHMO', 'object'

In [14]:
# Get the last message from the sender
last_msg = messages.get_last_text_message_by_sender("assistant")
if last_msg:
    print(f"Last Message: {last_msg.text.value}")

Last Message: I have created the bar chart for the operating profit of the companies. You can download it using the link below:

[Download the bar chart](sandbox:/mnt/data/operating_profit_bar_chart.png)


In [20]:
for file_path_annotation in messages.file_path_annotations:
    print(f"File Paths: {file_path_annotation}")
    file_name = f"{file_path_annotation.file_path.file_id}_text_file.txt"
    project_client.agents.save_file(file_id=file_path_annotation.file_path.file_id, file_name=file_name)
    print(f"Saved text file to: {os.getcwd()}\\{file_name}")

File Paths: {'type': 'file_path', 'text': 'sandbox:/mnt/data/operating_profit_bar_chart.png', 'start_index': 140, 'end_index': 188, 'file_path': {'file_id': 'assistant-H1xD6bD84ahBz155QIPDZl5v'}}
Saved text file to: E:\Users\mauromi\source\git_repos\aaas\assistant-H1xD6bD84ahBz155QIPDZl5v_text_file.txt


In [21]:
# Generate an image file for the bar chart
for image_content in messages.image_contents:
    print(f"Image content: {image_content}")
    file_name = f"{image_content.image_file.file_id}_image_file.png"
    project_client.agents.save_file(file_id=image_content.image_file.file_id, file_name=file_name)
    print(f"Saved image file to: {os.getcwd()}\\{file_name}")

Image content: {'type': 'image_file', 'image_file': {'file_id': 'assistant-oyhGeEMWiaxncjaak161lDMQ'}}
Saved image file to: E:\Users\mauromi\source\git_repos\aaas\assistant-oyhGeEMWiaxncjaak161lDMQ_image_file.png


In [22]:
# Delete all agents
for pca in project_client.agents.list_agents()['data']:
    print(f"Deleting agent {pca.name} ({pca.id})...")
    project_client.agents.delete_agent(pca.id)

Deleting agent my-agent (asst_RFBLqCOc6t5wct92nS7bIqhK)...


# HIC SUNT LEONES